In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import chain

# Graph construction
m = 4
n = 2**m - 1
Gm = [i for i in range(1, n)]
z = [(e, list(filter(lambda x: (x & e == 0) and (e > x), Gm))) for e in Gm]
E = []
for i in z:
    a, b = i
    for j in b:
        E.append((a, j))

# Convert edges to graph representation
def edges_to_graph(edges):
    graph = {}
    for u, v in edges:
        if u not in graph:
            graph[u] = set()
        if v not in graph:
            graph[v] = set()
        graph[u].add(v)
        graph[v].add(u)  # Since the graph is undirected
    return graph

# Example usage
edges = E
graph = edges_to_graph(edges)

# Modified degeneracy ordering function
def degeneracy_ordering(graph):
    """
    Compute degeneracy ordering of the graph, prioritizing vertices with the highest degree first.
    """
    order = []
    degree_dict = {v: len(graph[v]) for v in graph}  # Compute degrees
    remaining_vertices = set(graph.keys())

    while remaining_vertices:
        # Select the vertex with the highest degree
        v = max(remaining_vertices, key=lambda x: degree_dict[x])
        order.append(v)
        remaining_vertices.remove(v)

        # Update the degrees of its neighbors
        for neighbor in graph[v]:
            if neighbor in remaining_vertices:
                degree_dict[neighbor] -= 1

    return order

# Expand clique function
def expand_clique(v, candidates, graph):
    """
    Expands the current clique by keeping only those neighbors of v.
    """
    return candidates & graph[v]

# Restore candidates function
def restore_candidates(candidates, removed):
    """
    Restores previously removed candidates for backtracking.
    """
    candidates.update(removed)

# Recursive function for maximal clique enumeration
def mce_recursive(candidates, current_clique, cliques, graph, seen_cliques, clique_count):
    """
    Recursive function for Maximal Clique Enumeration.
    """
    if not candidates:
        clique_tuple = tuple(sorted(current_clique))
        if clique_tuple not in seen_cliques:
            seen_cliques.add(clique_tuple)
            cliques.append(set(current_clique))
            clique_count[0] += 1
        return

    pivot = max(candidates, key=lambda x: len(graph[x] & candidates))
    non_neighbors = candidates - graph[pivot]
    removed = set()

    for v in non_neighbors:
        if v not in candidates:
            continue
        new_candidates = expand_clique(v, candidates, graph)
        current_clique.add(v)
        try:
            mce_recursive(new_candidates, current_clique, cliques, graph, seen_cliques, clique_count)
        finally:
            current_clique.remove(v)
        removed.add(v)
    restore_candidates(candidates, removed)

# Main function for maximal clique enumeration
def maximal_clique_enumeration(graph):
    """
    Main function to enumerate all maximal cliques with improved efficiency.
    """
    cliques = []
    seen_cliques = set()
    clique_count = [0]
    order = degeneracy_ordering(graph)
    graph = {v: set(graph[v]) for v in graph}  # Ensure fast set operations

    for v in order:
        candidates = graph[v].copy()
        mce_recursive(candidates, {v}, cliques, graph, seen_cliques, clique_count)

    print(f"Total maximal cliques: {clique_count[0]}")
    return cliques




In [3]:

%%time
maximal_clique_enumeration(graph)

Total maximal cliques: 14
CPU times: total: 0 ns
Wall time: 3 ms


[{1, 2, 4, 8},
 {1, 2, 12},
 {1, 4, 10},
 {1, 6, 8},
 {1, 14},
 {2, 13},
 {2, 5, 8},
 {2, 4, 9},
 {4, 11},
 {3, 4, 8},
 {7, 8},
 {3, 12},
 {5, 10},
 {6, 9}]